In [1]:
import sys
sys.path.append('..')

In [ ]:
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy import stats

In [ ]:
from DomainPrediction.utils import helper
from DomainPrediction.eval import metrics
from DomainPrediction.al import top_model as topmodel
from DomainPrediction.al.embeddings import one_hot_encode
from DomainPrediction.protein.base import BaseProtein
from DomainPrediction.utils.constants import *

In [ ]:
sys.path.append('../../esm')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein, LogitsConfig, GenerationConfig, ESMProteinTensor
from esm.utils.structure.protein_chain import ProteinChain

In [ ]:
from esm.utils.generation import (
    _stack_protein_tensors,
)


### DPO Class

In [ ]:
class DPOTrainer:
    def __init__(self, model, model_ref, device='cpu') -> None:
        self.device = device

        # if self.device == 'gpu':
        #     self.model = model.to("cuda:0")
        #     self.model_ref = model_ref.to("cuda:1")
        # else:
        #     self.model = model.to("cpu")
        #     self.model_ref = model_ref.to("cpu")

        self.model = model
        self.model_ref =  model_ref

        self.model_ref.eval()
        for pm in self.model_ref.parameters():
            pm.requires_grad = False

        for name, param in self.model.named_parameters():
            if name in [
                "encoder.sequence_embed.weight", 
                "output_heads.sequence_head.0.weight", 
                "output_heads.sequence_head.0.bias",
                "output_heads.sequence_head.2.weight",
                "output_heads.sequence_head.2.bias",
                "output_heads.sequence_head.3.weight",
                "output_heads.sequence_head.3.bias"
            ]:
                param.requires_grad = True
            else:
                param.requires_grad = False

        ## need to add peft here ? Do we ?
    
    def print_trainable_parameters(self, model):
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
        )

    def generate_sequences(self, sequence_prompt, structure_prompt, size, ref=False):
        sequence_prediction_config = GenerationConfig(
            track="sequence", 
            num_steps=sequence_prompt.count("_") // 2, 
            temperature=0.5
        )
        esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=structure_prompt)
        
        generated_proteins = []
        for _ in tqdm(range(size)):
            generated_protein = self.model.generate(esm_protein, sequence_prediction_config)
            generated_proteins.append(generated_protein.sequence)

        return generated_proteins
    
    def create_dataset(self, sequence_prompt, structure_prompt, mask_positions, size):

        generated_sequences = self.generate_sequences(sequence_prompt, structure_prompt, size)

        dataset = []
        for _seq in generated_sequences:
            dataset.append({
                'generated_sequence': _seq,
                'masked_sequence': ''.join([s for _i, s in enumerate(_seq) if _i in mask_positions]),
                'masked_positions': mask_positions, ## These are the same
                'sequence_prompt': sequence_prompt, ## These are the same
                'structure_prompt': structure_prompt, ## These are the same   
            })

        for item in dataset:
            item['property'] = self.__get_n_mutations(item['masked_sequence'])
        
        return dataset
    
    def get_mask_positions(self, sequence_prompt, mask_var='_'):
        positions = []
        for i in range(len(sequence_prompt)):
            if sequence_prompt[i] == mask_var:
                positions.append(i)

        return np.array(positions)
    
    def __get_n_mutations(self, sequence):
        base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
        assert len(base) == len(sequence)

        count = 0
        for aa1, aa2 in zip(base, sequence):
            if aa1 != aa2:
                count += 1

        return -1*count
    
    # def get_property_batch(self, batch):
    #     for item in batch:
    #         item['property'] = self.__get_n_mutations(item['masked_sequence'])

    #     return batch
    
    def get_PLL(self, item, ref=False):
        '''
            We always feed in the sequence prompt
            We get Log Prob(A_C) -  entire T region is masked
        '''
        proteins = [ESMProtein(sequence=item['sequence_prompt'], 
                        coordinates=item['structure_prompt'])]
        
        if ref:
            input_tokens = [self.model_ref.encode(protein) for protein in proteins]
            generated_tokens = self.model_ref.encode(ESMProtein(sequence=item['generated_sequence'], 
                                                        coordinates=item['structure_prompt']))
        else:
            input_tokens = [self.model.encode(protein) for protein in proteins]
            generated_tokens = self.model.encode(ESMProtein(sequence=item['generated_sequence'], 
                                                        coordinates=item['structure_prompt']))
        
        
        devices = set([t.device for t in input_tokens])
        if len(devices) > 1:
                raise AttributeError(f"Input tokens on multiple devices {devices}")
        sequence_lengths = [len(tokens) for tokens in input_tokens]
        assert len(set(sequence_lengths)) == 1
        
        if ref:
            batched_tokens = _stack_protein_tensors(
                    input_tokens, sequence_lengths, self.model_ref.tokenizers, self.model_ref.device
                )
        else:
            batched_tokens = _stack_protein_tensors(
                    input_tokens, sequence_lengths, self.model.tokenizers, self.model.device
                )
        
        if batched_tokens.coordinates is None:
            per_res_plddt = None
        else:
            # 1.0 if all coordinates at specific indices have valid non-nan values.
            per_res_plddt = batched_tokens.coordinates.isfinite().all(dim=-1).any(dim=-1).float()

        if ref:
            assert model_ref.device == batched_tokens.device
            with (torch.no_grad(),
                torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
                output = self.model_ref.forward(
                        sequence_tokens=batched_tokens.sequence,
                        structure_tokens=batched_tokens.structure,
                        ss8_tokens=batched_tokens.secondary_structure,
                        sasa_tokens=batched_tokens.sasa,
                        function_tokens=batched_tokens.function,
                        residue_annotation_tokens=batched_tokens.residue_annotations,
                        average_plddt=torch.tensor(1.0, device=batched_tokens.device),
                        per_res_plddt=per_res_plddt,
                        structure_coords=batched_tokens.coordinates,
                        chain_id=None,
                        sequence_id=None,
                )
        else:
            assert model.device == batched_tokens.device
            with (torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
                output = self.model.forward(
                        sequence_tokens=batched_tokens.sequence,
                        structure_tokens=batched_tokens.structure,
                        ss8_tokens=batched_tokens.secondary_structure,
                        sasa_tokens=batched_tokens.sasa,
                        function_tokens=batched_tokens.function,
                        residue_annotation_tokens=batched_tokens.residue_annotations,
                        average_plddt=torch.tensor(1.0, device=batched_tokens.device),
                        per_res_plddt=per_res_plddt,
                        structure_coords=batched_tokens.coordinates,
                        chain_id=None,
                        sequence_id=None,
                )
        
        log_prob = torch.log_softmax(output.sequence_logits, dim=-1)

        masked_log_prob = log_prob[:, item['masked_positions'] + 1, generated_tokens.sequence[item['masked_positions'] + 1]]
        pll = torch.sum(masked_log_prob)

        # print(item['masked_sequence'])
        print(generated_tokens.sequence[item['masked_positions'] + 1])
        print(masked_log_prob)
        # print(pll)

        return pll
         
    def dpo_paired_loss(self, batch):

        pll_policy = []
        pll_ref = []
        for item in batch:
            pll_policy.append(self.get_PLL(item))
            pll_ref.append(self.get_PLL(item, ref=True))
            print(item['property'])

        print([x['property'] for x in batch])
        print(pll_policy)
        print(pll_ref)

        pll_ref = [x.to(self.model.device) for x in pll_ref]

        print(pll_ref)

        batch_size = len(batch)
        assert batch_size > 1
        beta = 0.01

        loss = []

        pairs_debug = []
        for i in range(batch_size-1):
            for j in range(1, batch_size):
                if batch[i]['property'] > batch[j]['property'] + 0:
                    loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i] - pll_ref[i]) - (pll_policy[j] - pll_ref[j])) )
                    # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i]) - (pll_policy[j])) )
                    loss.append(loss_item)

                    pairs_debug.append((batch[i]['property'], batch[j]['property'],
                                        (pll_policy[i] - pll_ref[i]).item(), (pll_policy[j] - pll_ref[j]).item(),
                                        loss_item.item()))
                    
                elif batch[j]['property'] > batch[i]['property'] + 0:
                    loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j] - pll_ref[j]) - (pll_policy[i] - pll_ref[i])) )
                    # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j]) - (pll_policy[i])) )
                    loss.append(loss_item)

                    pairs_debug.append((batch[j]['property'], batch[i]['property'],
                                        (pll_policy[j] - pll_ref[j]).item(), (pll_policy[i] - pll_ref[i]).item(),
                                        loss_item.item()))
                    
        for _pair in pairs_debug:
            print(f'pair :{_pair}')

        if len(loss) > 0:
            loss = torch.mean(torch.stack(loss))
        else:
            loss = torch.tensor(0.).to(self.model.device)
                    
        return loss
 
    def train(self, sequence_prompt, structure_prompt):

        self.model.train()

        self.print_trainable_parameters(self.model)
        self.print_trainable_parameters(self.model_ref)

        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=1e-3,
            betas=(0.9, 0.98),
            eps=1e-8,
            weight_decay=0.1,
        )

        mask_positions = self.get_mask_positions(sequence_prompt=sequence_prompt)

        dataset = self.create_dataset(sequence_prompt, structure_prompt,
                                      mask_positions, 10)
        
        torch.cuda.empty_cache()
        
        return dataset

        # epochs = 100
        # batch_size = 1
        # for epoch in range(epochs):
        #     optimizer.zero_grad()

        #     # A, C = sequence_prompt.split('_'*115)
        #     # base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
        #     # batch_dataset.append({
        #     #     'generated_sequence': A+base+C,
        #     #     'masked_sequence': base,
        #     #     'masked_positions': mask_positions, ## These are the same
        #     #     'sequence_prompt': sequence_prompt, ## These are the same
        #     #     'structure_prompt': structure_prompt ## These are the same  
        #     # })

        #     batch_dataset = self.get_property_batch(batch_dataset)
            
        #     print(f'epoch {epoch}: calculating loss')
        #     loss = self.dpo_paired_loss(batch_dataset)
        #     print(f'loss {loss.item()}')

        #     if loss != 0:
        #         print(f'epoch {epoch}: optimizer step')
        #         loss.backward()
        #         optimizer.step()

        #     torch.cuda.empty_cache()

In [ ]:
protein = ProteinChain.from_pdb('../../Data/gxps/gxps_ATC_AF.pdb')

sequence_prompt = ''.join([protein[i].sequence if i in A_gxps_atc + C_gxps_atc else '_' for i in range(len(protein))])
structure_prompt = torch.tensor(protein.atom37_positions)

# structure_prompt = torch.tensor(protein.atom37_positions)[T_gxps_atc[0]-50:T_gxps_atc[-1]+50]
# sequence_prompt = ''.join([s for i, s in enumerate(sequence_prompt) if i in list(range(T_gxps_atc[0]-50,T_gxps_atc[-1]+50))])

In [ ]:
len(sequence_prompt), sequence_prompt

In [ ]:
len(sequence_prompt) == structure_prompt.shape[0]

In [ ]:
## Write script to check the availability of the GPUs and no of GPUs

In [ ]:
model = ESM3.from_pretrained("esm3_sm_open_v1", device=torch.device('cuda:0'))

In [ ]:
model_ref = ESM3.from_pretrained("esm3_sm_open_v1", torch.device('cuda:1'))

In [ ]:
dpo_trainer = DPOTrainer(model, model_ref, device='gpu')

In [ ]:
os.environ["DISABLE_ITERATIVE_SAMPLING_TQDM"] = "True"

In [ ]:
dataset = dpo_trainer.train(sequence_prompt=sequence_prompt, structure_prompt=None)

In [ ]:
dataset

In [ ]:
def create_pairs(dataset):
    
    pair_dataset = []
    for item_1 in batch[:-1]:
        for item_2 in batch[1:]:
            if item_1['property'] < item_2['property'] and item_1['log_likelihood'] < item_2['log_likelihood']:
                pair_dataset.append((item_1, item_2))
            elif item_1['property'] > item_2['property'] and item_1['log_likelihood'] > item_2['log_likelihood']:
                pair_dataset.append((item_2, item_1))

    # _, ax = plt.subplots(1, 2, figsize=(7, 3), layout='constrained')
    # ax[0].hist([[item[0]['property'] for item in pair_dataset], 
    #             [item[1]['property'] for item in pair_dataset]], label=['pos', 'neg'])
    # ax[1].hist([[item[0]['log_likelihood'] for item in pair_dataset],
    #             [item[1]['log_likelihood'] for item in pair_dataset]], label=['pos', 'neg'])
    # ax[0].legend()
    # ax[1].legend()
    # plt.show()

    return pair_dataset